# ASR - An experimental notebook on supervised Neural Networks.


## Ideas

- symantic web technology, micro applications (combine the approaches)
- the role of authors, citations
- sensitivity 1 doesn't work. (multiple raters, different expertise)
- citation networks
- use of rules how to interpreted the text. 


## Remarks

- Covidence and Rayyan (software)
- 



# Systematic review

Based on: https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py

> This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classification of newsgroup messages into 20 different categories).
GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)
20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html

## Evaluation Criteria

- minimize number of papers
- max # of fn =1
- threshhold >0


## Related links
https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/
https://machinelearningmastery.com/best-practices-document-classification-deep-learning/
https://www.quora.com/What-deep-learning-method-to-use-to-classify-text-files

Import dependencies

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function

# CPython dependencies
import os
import sys

# external dependencies
import numpy as np
import pandas as pd
import sklearn
import keras_metrics
from nltk.corpus import stopwords
from prettytable import PrettyTable
from numpy import cumsum
from matplotlib import pyplot
from datetime import datetime
from scipy import stats
from sklearn.model_selection import train_test_split

# keras dependencies
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM
from keras.layers import Conv1D, MaxPooling1D, Embedding, Bidirectional
from keras.models import Model
from keras import metrics
from keras.callbacks import TensorBoard

# project dependencies
sys.path.insert(0, os.path.join('..', 'python'))

from utils import load_ptsd_data
from config import LOG_DIR, OUTPUT_DIR, DATA_DIR, GLOVE_DIR

## Data

Basic data related variables.

In [ ]:
# the target variable
TARGET_VARIABLE = "included_final" # "included_ats" (after title screening)

Read data into memory.

In [ ]:
texts, labels = load_ptsd_data()
print('{} articles in SR, {} included'.format(len(texts), sum(labels)))

## Word tokenizing

Apply vectorization to texts.

In [ ]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000

In [ ]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# get the word index
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='post', truncating='post')
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

## Create training and validation set
Split the dataset into a training and a validation set. Use SciKit-learn for this. 
We decided to train our model on a small dataset (around 500), to make the environment more simulated to the active learning approach where we have small number of labeled data points. 

In [ ]:

def split_data(validation_split, added_positives):
    x_train, x_val, y_train, y_val = train_test_split(
        data,
        labels,
        test_size=validation_split,
        random_state=2018,
        stratify=labels
    )

    # add added_positives positive paper to training dataset
    if added_positives>0 :
        positive_indx = np.where(y_val[:,1]==1)[0]
        x_train = np.vstack((x_train, x_val[positive_indx[0:added_positives]]))
        y_train = np.vstack((y_train,y_val[positive_indx[0:added_positives]]))

        x_val = np.delete(x_val, positive_indx[0:added_positives],0)
        y_val = np.delete(y_val, positive_indx[0:added_positives],0)

    return (x_train, x_val, y_train, y_val)

x_train, x_val, y_train, y_val = split_data(0.2, 0)

print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
print("included papers in train dataset:",(y_train[:,1]==1).sum())
print("included papers in test dataset:",(y_val[:,1]==1).sum())


## Set up an embedding layer

We make use of a 300 dimensional embedding layer for the RNN models. This embedding layer is based on pretrained Word2Vec models. In this step we match our vocabulary with the vocabulary of the pretrained models. Thereafter, we embed the result in a Keras Embedding layer. 

In [ ]:
# first, build index mapping words in the embeddings set
# to their embedding vector

# set dimensions
EMBEDDING_DIM = 300
pretrained_word2vec_fp = os.path.join(GLOVE_DIR, 'wiki.en.vec')

embeddings_index = {}
with open(pretrained_word2vec_fp, encoding='utf8') as f:
    for line in f:
        
        values = line.split()
        split_on_i = len(values) - EMBEDDING_DIM
        word = ' '.join(values[0:split_on_i])
        coefs = np.asarray(values[split_on_i:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
            continue
        
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
                
print('Shape of embedding matrix: ', embedding_matrix.shape)

The following function is use to create a embedding layer. 

In [ ]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
def build_embedding(weights):
    return Embedding(num_words,
                                EMBEDDING_DIM,
                                weights=[weights],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)

embedding_layer = build_embedding(embedding_matrix)

## Models

This section contains models to use later on. Focusing on Convolutional and LSTM models. 

### LSTM

In [ ]:
def get_lstm_model(backwards, dropout, neurons,optimizer ):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    x = LSTM(neurons,input_shape=(MAX_SEQUENCE_LENGTH,),  go_backwards=backwards, dropout=dropout)(embedded_sequences)
    x = Dense(128,activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model_lstm = Model(inputs=sequence_input, outputs=output)

    model_lstm.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc',keras_metrics.recall()])

    model_lstm.summary()
    return model_lstm

def get_bi_lstm_model(mode, dropout, neurons, optimizer):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    x = Bidirectional(LSTM(neurons,input_shape=(MAX_SEQUENCE_LENGTH,), dropout=dropout),merge_mode= mode)(embedded_sequences)
    x = Dense(128, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model_lstm = Model(inputs=sequence_input, outputs=output)

    model_lstm.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc',keras_metrics.recall()])

    model_lstm.summary()
    return model_lstm


### Convnet

In [ ]:
# train a 1D convnet with global maxpooling
def build_convnet():
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    x = Conv1D(128, 5, activation='relu')(embedded_sequences)
    x = MaxPooling1D(5)(x)
    x = Conv1D(128, 5, activation='relu')(x)
    x = MaxPooling1D(5)(x)
    x = Conv1D(128, 5, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    preds = Dense(2, activation='softmax')(x)

    model_convnet = Model(inputs=sequence_input, outputs=preds)

    model_convnet.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc',keras_metrics.recall()])
    return model_convnet
    
    
model_convnet = build_convnet()

## Train the model

### calculating class weights

In [ ]:
weights = [1/y_val[:, 0].mean(), 1/y_val[:, 1].mean()]

### Visualize model scores with TensorBoard

In [ ]:
def save_run(sub_dir=None):
    
    if sub_dir is not None:
        log_dir = os.path.join(LOG_DIR, sub_dir)
    else:
        log_dir = LOG_DIR
    
    return TensorBoard(log_dir=log_dir, histogram_freq=0,
                       write_graph=True, write_images=True)
    


In [ ]:
def train_model(model, epoch_no):
    
    start=datetime.now()
    hist = model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=epoch_no,
            validation_data=(x_val, y_val),
            shuffle=True,
            class_weight=weights,
            verbose = 0
        )  

    runtime=datetime.now() - start
    
    loss = hist.history['loss'][0]    
    return (model,loss, runtime)
 

def get_scores(model, threshhold):
    prediction = model.predict(x_val)
    y_classes = ( [0 if x < threshhold else 1 for x in prediction[:,1]])
    y_value = y_val.argmax(axis=1)
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_value,y_classes).ravel()
    return (tn, fp, fn, tp)


def get_pred(model):
    prediction = model.predict(x_val)
    return prediction


def get_scores_pred(pred, threshhold):
    y_classes = ( [0 if x < threshhold else 1 for x in pred[:,1]])
    y_value = y_val.argmax(axis=1)
    (tn, fp, fn, tp) = sklearn.metrics.confusion_matrix(y_value,y_classes).ravel()
    return (tn, fp, fn, tp)


### Compare Various LSTM models
Comparing LSTM forward, LSTM backward and Bi-directional LSTM

####Trained Dataset : 4000 papers

In [ ]:
epochs = 10
BATCH_SIZE = 128
dropout=0
neurons=10
optimizer='rmsprop'

In [ ]:
# lstm forwards
print('lstm forwards')
model = get_lstm_model(False,dropout,neurons,optimizer)
(trained_model,loss, runtime) = train_model(model, epochs)
forward_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

In [ ]:
print(stats.describe(forward_pred[:,1]))
threshhold=0.006
(tn, fp, fn, tp) = get_scores_pred(forward_pred, threshhold)
forward_scores =(tn, fp, fn, tp)
print(forward_scores)
forward_result = ('forward',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

In [ ]:
# lstm backwards
print('lstm_back')
model = get_lstm_model(True,dropout,neurons,optimizer)
(trained_model,loss, runtime) = train_model(model, epochs)
backward_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

In [ ]:
print(stats.describe(backward_pred[:,1]))
threshhold=0.001
(tn, fp, fn, tp) = get_scores_pred(backward_pred, threshhold)
backward_scores = (tn, fp, fn, tp)
print(tn, fp, fn, tp)
backward_result = ('backward',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

In [ ]:
# bidirectional concat
print('bidirectional concat')
model = get_bi_lstm_model('concat', 0,neurons,optimizer)
(trained_model,loss, runtime) = train_model(model, epochs)
bilstm_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

In [ ]:
print(stats.describe(bilstm_pred[:,1]))
threshhold=0.004
(tn, fp, fn, tp) = get_scores_pred(bilstm_pred, threshhold)
bi_scores = (tn, fp, fn, tp)
print(tn, fp, fn, tp)
bi_result = ('bidirectional',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

# # line plot of results
# results.plot()
# pyplot.show()

In [ ]:
backward_result

### Result
As it can be seen in the following table, for 5 and 10 epochs, without any dropout, backward LSTM achieve the best result. For each model, we found the best threshhold to meet our evaluation criteria.
For the rest of our analysis we select backward LSTM since it achieve the best results in a shorter amount of time.

In [ ]:
t = PrettyTable(['Model','threshhold','TN', 'FP', 'FN','TP','Loss','RunTime (sec)'])

t.add_row(forward_result)
t.add_row(backward_result)
t.add_row(bi_result)

print(t)


Now we start to reduce the number of trained datasets
####  Trained Dataset : 500 papers, included papers: 20 

In [ ]:
## train model on 500 datapoints
x_train, x_val, y_train, y_val = split_data(0.9, 16)

print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
print("included papers in train dataset:",(y_train[:,1]==1).sum())
print("included papers in test dataset:",(y_val[:,1]==1).sum())

In [ ]:
epochs = 10
BATCH_SIZE = 128
dropout=0
neurons=10
optimizer='rmsprop'

In [ ]:
# lstm backwards
print('lstm_back')
model = get_lstm_model(True,dropout,neurons,optimizer)
(trained_model,loss500, runtime500) = train_model(model, epochs)
backward_pred500 = get_pred(trained_model)
print('loss:' ,loss500,'runtime:', runtime500.total_seconds())

In [ ]:
print(stats.describe(backward_pred500[:,1]))
threshhold=0.03
(tn, fp, fn, tp) = get_scores_pred(backward_pred500, threshhold)
backward_scores500 = (tn, fp, fn, tp)
print(backward_scores500)
backward_result500 = ('500 papers',threshhold,tn, fp, fn, tp, loss500, runtime500.total_seconds())
print(backward_result500)

####  Trained Dataset : 100 papers, included papers: 20 

In [ ]:
## train model on 100 datapoints
x_train, x_val, y_train, y_val = split_data(0.98, 20)

print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
print("included papers in train dataset:",(y_train[:,1]==1).sum())
print("included papers in test dataset:",(y_val[:,1]==1).sum())

In [ ]:
epochs = 10
BATCH_SIZE = 128
dropout=0.25
neurons=10
optimizer='rmsprop'

In [ ]:
# lstm backwards
print('lstm_back')
model = get_lstm_model(True,dropout,neurons,optimizer)
(trained_model,loss100, runtime100) = train_model(model, epochs)
backward_pred100 = get_pred(trained_model)
print('loss:' ,loss100,'runtime:', runtime100.total_seconds())

In [ ]:
print(stats.describe(backward_pred100[:,1]))
threshhold=0.15
(tn, fp, fn, tp) = get_scores_pred(backward_pred100, threshhold)
backward_scores100 = (tn, fp, fn, tp)
print(backward_scores100)
backward_result100 = ('100 papers',threshhold,tn, fp, fn, tp, loss100, runtime100.total_seconds())
print(backward_result100)

####  Trained Dataset : 50 papers, included papers: 20 

In [ ]:
## train model on 100 datapoints
x_train, x_val, y_train, y_val = split_data(0.99, 10)

print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
print("included papers in train dataset:",(y_train[:,1]==1).sum())
print("included papers in test dataset:",(y_val[:,1]==1).sum())

In [ ]:
epochs = 10
BATCH_SIZE = 128
dropout=0.3
neurons=10
optimizer='rmsprop'

In [ ]:
# lstm backwards
print('lstm_back')
model = get_lstm_model(True,dropout,neurons,optimizer)
(trained_model,loss50, runtime50) = train_model(model, epochs)
backward_pred50 = get_pred(trained_model)
print('loss:' ,loss50,'runtime:', runtime50.total_seconds())

In [ ]:
print(stats.describe(backward_pred100[:,1]))
threshhold=0.14
(tn, fp, fn, tp) = get_scores_pred(backward_pred50, threshhold)
backward_scores50 = (tn, fp, fn, tp)
print(backward_scores50)
backward_result50 = ('50 papers',threshhold,tn, fp, fn, tp, loss50, runtime50.total_seconds())
print(backward_result50)

In [ ]:
backward_result4000 =('4000 papaers',*backward_result[1:]) 
col_labels = ['train_dataset','threshhold','TN', 'FP', 'FN','TP','Loss','RunTime (sec)']
data_rows = [backward_result4000,backward_result500,backward_result100,backward_result50]
df = pd.DataFrame.from_records(data_rows, columns=col_labels)

df['FP_rate']=df.loc[:,'FP']/(df.loc[:,'FP']+df.loc[:,'TN'])
df['Recall']=df.loc[:,'TP']/(df.loc[:,'TP']+df.loc[:,'FN'])
print(df)

In [ ]:
tt = PrettyTable(['#train dataset','threshhold','TN', 'FP', 'FN','TP','Loss','RunTime (sec)'])

tt.add_row(backward_result4000)
tt.add_row(backward_result500)
tt.add_row(backward_result100)
tt.add_row(backward_result50)

print(tt)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_fp_rate = pd.DataFrame([['FP_rate',4000,0.077],['FP_rate',500,0.854],['FP_rate',100,0.904],['FP_rate',50,0.846],
                   ['Recall',4000,0.875],['Recall',500,0.950],['Recall',100,0.947],['Recall',50,0.966]],
                    columns=['score','trained_dataset','val'])

df_fp_rate.pivot("trained_dataset", "score", "val").plot(kind='barh')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2)
plt.title('Model scores based on size of the training dataset')
plt.show()

# df_tn_fp = pd.DataFrame([['FP','4000',78],['FP','500',3876],['FP','100',4467],['FP','50',4223],
#                    ['TN','4000',930],['TN','500',678],['TN','100',470],['TN','50',768]],columns=['score','trained_dataset','val'])


# df_tn_fp.pivot("trained_dataset", "score", "val").plot(kind='bar')

# plt.show()





We added 15 positive cases to training, it causes that model label all data as positive. Now I try with stratified samples.

### Convnet

#### Training dataset size = 4000

In [ ]:
x_train, x_val, y_train, y_val = split_data(0.2, 0)

print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
print("included papers in train dataset:",(y_train[:,1]==1).sum())
print("included papers in test dataset:",(y_val[:,1]==1).sum())


In [ ]:
epochs = 10
BATCH_SIZE = 128
dropout=0.3
neurons=10
optimizer='rmsprop'

In [ ]:
# convnet
print('convnet')
model = build_convnet()
(trained_model,loss, runtime) = train_model(model, epochs)
forward_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

In [ ]:
print(stats.describe(forward_pred[:,1]))
threshhold=0.00000001
(tn, fp, fn, tp) = get_scores_pred(forward_pred, threshhold)
cnn_score = (tn, fp, fn, tp)
print(cnn_score)
cnn_result = ('4000 papers',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())
print(cnn_result)